# Correctionlib to text

In [1]:
import json
import os,sys
import pandas as pd
import numpy as np
import correctionlib

### Setting up some global parameters and functions

In [2]:
objectdict = {
    'Electron':{
        'basedir':'POG/EGM/',
        'jsonfile': 'electron.json',
        'corrections':{
            'UL-Electron-ID-SF':  'electron_id_sf.txt'
        },
        'outdir':'corrections/electronsf'
    },
    'Muon':{
        'basedir':'POG/MUO/',
        'jsonfile': 'muon_Z_v2.json',
        'corrections':{
            'NUM_MediumID_DEN_genTracks':  'muon_id_sf.txt',
            'NUM_TightRelIso_DEN_MediumID':'muon_iso_sf.txt'
        },
        'outdir':'corrections/muonsf'
    },
    'Jet-JEC':{
        'basedir':'POG/JME/',
        'jsonfile': 'jet_jerc.json',
        'corrections':{
            'Summer19UL18_V5_MC_Total_AK4PFchs': 'jet_jec_sf.txt',
        },
        'outdir':'corrections/jetsf'
    },
    'Jet-JER':{
        'basedir':'POG/JME/',
        'jsonfile': 'jet_jerc.json',
        'corrections':{
            'Summer19UL18_JRV2_MC_ScaleFactor_AK4PFchs': 'jet_jer_sf.txt',
        },
        'outdir':'corrections/jetsf'
    },
    'Jet-pT':{
        'basedir':'POG/JME/',
        'jsonfile': 'jet_jerc.json',
        'corrections':{
            'Summer19UL18_JRV2_MC_PtResolution_AK4PFchs': 'jet_ptres_sf.txt',
        },
        'outdir':'corrections/jetsf'
    },
    'bJet':{
        'basedir':'POG/BTV/',
        'jsonfile': 'btagging.json',
        'corrections':{
            'deepJet_mujets': 'bjet_mujets_and_incl_eff.txt',
            'deepJet_comb': 'bjet_comb_and_incl_eff.txt'
        },
        'outdir':'corrections/bjeteff'
    }
}

def warning(text):
    text = '\033[031mWarning! '+text+'\033[0m'
    print(text)

#### Extracting electron scale-factors in pT-eta bins

In [3]:
def parse_electron_sf(filename, correction_name, campaign):
    scale_factors = []
    with open(filename, "r") as f: json_data = json.load(f)

    pt_edges = None
    eta_edges = None

    for item in json_data['corrections']:
        #Each item is a dict
        if item['name'] != correction_name: continue
        
        content = item['data']['content']
        for obj in content:
            ### campaign name
            if obj['key'] not in campaign: continue
            print('Extracting data for: '+obj['key'])
            
            subcontent = obj['value']['content']
            for subobj in subcontent:
                ### sf type
                if subobj['key'] != 'sf':continue
                print('Extracting data for:'+subobj['key'])

                subsubcontent = subobj['value']['content']
                for subsubobj in subsubcontent:
                    ### Working point
                    if subsubobj['key'] != 'Medium': continue
                    print('Extracting data for '+subsubobj['key']+' WP')

                    edges = subsubobj['value']['edges']
                    eta_edges = edges[0]
                    pt_edges  = edges[1]
                    print('Edges extracted!')

    # Now that the binning is calculated,
    #print(pt_edges)
    #print(eta_edges)
    correction_set = correctionlib.CorrectionSet.from_file(filename)
    correction = correction_set[correction_name]
    MAX_PT = 1500
    MIN_ETA = -2.5
    MAX_ETA = 2.5
    
    #Given the pt and eta edges, loop over their midvalues.
    for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
        for pt_low, pt_high in zip(pt_edges[:-1], pt_edges[1:]):

            if not np.isfinite(eta_low):  eta_low  = MIN_ETA if eta_low == -np.inf else MAX_ETA
            if not np.isfinite(eta_high): eta_high = MIN_ETA if eta_high == -np.inf else MAX_ETA
            if not np.isfinite(pt_low):     pt_low = MAX_PT if pt_low == np.inf else 0
            if not np.isfinite(pt_high):   pt_high = MAX_PT if pt_high == np.inf else 0 
            
            eta = (eta_low + eta_high) / 2
            pt  = (pt_low  + pt_high) / 2
            era = campaign.replace('_UL', '')
            wp = "Medium"
            values = [era, 'sf', wp, eta, pt]
            values_down = [era, 'sfdown', wp, eta, pt]
            values_up = [era, 'sfup', wp, eta, pt]
            sfdown = correction.evaluate(*values_down)
            sf = correction.evaluate(*values)
            sfup = correction.evaluate(*values_up)
            
            scale_factors.append({
                'campaign': campaign,
                'eta_low' : eta_low,
                'eta_high': eta_high, 
                'pt_low'  : pt_low,
                'pt_high' : pt_high,
                'sfdown'  : sfdown,
                'sf'      : sf,
                'sfup'    : sfup
            })

            #print(f"Scale factor: {sf}, sfdown: {sfdown}, sfup: {sfup}")
            #break ### ptbin
        #break ### etabin
   
    df = pd.DataFrame(scale_factors)
    print('Correctionlib evaluated and dataframe created.\n')
    return df

print('Function for electrons loaded.')

Function for electrons loaded.


#### Extracting muon scale-factors in pT-eta bins

In [4]:
def parse_muon_sf(filename, correction_name, campaign):
    scale_factors=[]
    with open(filename, "r") as f: json_data = json.load(f)

    pt_edges = None
    eta_edges = None

    for item in json_data['corrections']:
        #Each item is a dict
        if item['name'] != correction_name: continue

        content = item['data']['content']
        for obj in content:
            ### eta bins
            if obj['key'] not in campaign: continue
            print('Extracting data for: '+obj['key'])
            eta_edges = obj['value']['edges']
            subcontent = obj['value']['content']
            for subobj in subcontent:
                ### pt bins
                pt_edges = subobj['edges']
                print('Edges extracted!')
                break ### found pt endges

    # Now that the binning is calculated,
    #print(pt_edges)
    #print(eta_edges)
    correction_set = correctionlib.CorrectionSet.from_file(filename)
    correction = correction_set[correction_name]
    MAX_PT = 1500
    MIN_ETA = 0
    MAX_ETA = 2.5
    
    #Given the pt and eta edges, loop over their midvalues.
    for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
        for pt_low, pt_high in zip(pt_edges[:-1], pt_edges[1:]):
            if not np.isfinite(pt_high):   pt_high = MAX_PT if pt_high == np.inf else 0

            eta = (eta_low + eta_high) / 2
            pt  = (pt_low  + pt_high) / 2
            era = campaign
            values = [era, eta, pt, 'sf']
            values_down = [era, eta, pt, 'systdown']
            values_up = [era, eta, pt, 'systup']
            sfdown = correction.evaluate(*values_down)
            sf = correction.evaluate(*values)
            sfup = correction.evaluate(*values_up)
            
            scale_factors.append({
                'campaign': campaign,
                'eta_low' : eta_low,
                'eta_high': eta_high, 
                'pt_low'  : pt_low,
                'pt_high' : pt_high,
                'sfdown'  : sfdown,
                'sf'      : sf,
                'sfup'    : sfup
            })
                
    df = pd.DataFrame(scale_factors)
    return df

print('Function for muons loaded.')

Function for muons loaded.


#### Extracting JEC and JER in pT-eta bins

In [5]:
def parse_jet_jec_sf(filename, correction_name, campaign):
    scale_factors=[]
    with open(filename, "r") as f: json_data = json.load(f)

    pt_edges = None
    eta_edges = None

    match_found = False
    for item in json_data['corrections']:
        if item['name'] != correction_name: continue
        match_found = True
            
        #print(item['data'].keys())
        #print(item['data']['input'])
        #print(item['data']['edges'])
        eta_edges = item['data']['edges']
        content = item['data']['content']
        for obj in content:
            #print(obj.keys())
            #print(obj['input'])
            #print(obj['edges'])
            pt_edges = obj['edges']
            print('Edges extracted!')
            break

    if not match_found: warning(f'Not found: {correction_name}')
    if eta_edges ==None or pt_edges == None: return pd.DataFrame([])
    #print(f'JEC pT edges = {pt_edges}')
    #print(f'JEC eta edges = {eta_edges}')

    # Now that the binning is calculated,
    correction_set = correctionlib.CorrectionSet.from_file(filename)
    correction = correction_set[correction_name]
    
    #Given the pt and eta edges, loop over their midvalues.
    for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
        for pt_low, pt_high in zip(pt_edges[:-1], pt_edges[1:]):
            
            eta = (eta_low + eta_high) / 2
            pt  = (pt_low  + pt_high) / 2
            values = [eta, pt]
            unc = correction.evaluate(*values)
            sf = np.ones_like(unc)
            
            scale_factors.append({
                'campaign': campaign,
                'eta_low' : eta_low,
                'eta_high': eta_high, 
                'pt_low'  : pt_low,
                'pt_high' : pt_high,
                'sfdown'  : sf-unc,
                'sf'      : sf,
                'sfup'    : sf+unc
            })

    df = pd.DataFrame(scale_factors)
    return df

def parse_jet_jer_sf(filename, correction_name, campaign):

    scale_factors=[]
    with open(filename, "r") as f: json_data = json.load(f)

    pt_edges = None
    eta_edges = None

    match_found = False
    for item in json_data['corrections']:
        if item['name'] != correction_name: continue
        match_found = True

        #print(item['data'].keys())
        #print(item['data']['input'])
        #print(item['data']['edges'])
        eta_edges = item['data']['edges']
        print('Edges extracted!')

    if not match_found: warning(f'Not found: {correction_name}')
    #if eta_edges ==None or pt_edges == None: return pd.DataFrame([])
    #print(f'JER eta edges = {eta_edges}')
    
    correction_set = correctionlib.CorrectionSet.from_file(filename)
    correction = correction_set[correction_name]
    for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
        
        eta = (eta_low + eta_high) / 2
        values_nom = [eta, 'nom']
        values_up  = [eta, 'up']
        values_down = [eta, 'down']
        
        sf     = correction.evaluate(*values_nom)
        sfup   = correction.evaluate(*values_up)
        sfdown = correction.evaluate(*values_down)
        
        scale_factors.append({
            'campaign': campaign,
            'eta_low' : eta_low,
            'eta_high': eta_high,
            'sfdown'  : sfdown,
            'sf'      : sf,
            'sfup'    : sfup
        })
            
    df = pd.DataFrame(scale_factors)
    return df

def parse_jet_ptres_sf(filename, correction_name, campaign):

    scale_factors=[]
    with open(filename, "r") as f: json_data = json.load(f)

    # This is a function based correction. So pT ednges are not available.
    # In order to write in the text file, I pciked the same pT edges as in JEC and JER
    pt_edges = [9.0, 11.0, 13.5, 16.5, 19.5, 22.5, 26.0, 30.0, 34.5, 40.0,
                46.0, 52.5, 60.0, 69.0, 79.0, 90.5, 105.5, 123.5, 143.0,
                163.5, 185.0, 208.0, 232.5, 258.5, 286.0, 331.0, 396.0,
                468.5, 549.5, 639.0, 738.0, 847.5, 968.5, 1102.0, 1249.5,
                1412.0, 1590.5, 1787.0, 2003.0, 2241.0, 2503.0, 2790.5, 3107.0,
                3455.0, 3837.0, 4257.0, 4719.0, 5226.5, 5784.0, 6538.0]
    
    eta_edges = None
    rho_edges = None

    match_found = False
    for item in json_data['corrections']:
        if item['name'] != correction_name: continue
        match_found = True

        eta_edges = item['data']['edges']
        content = item['data']['content']
        for obj in content:
            rho_edges = obj['edges']
            print('Edges extracted!')
            break

    #print(f'pT-res eta edges = {eta_edges}')
    #print(f'pT-res rho edges = {rho_edges}')

    if not match_found: warning(f'Not found: {correction_name}')

    # Now that the binning is calculated,
    correction_set = correctionlib.CorrectionSet.from_file(filename)
    correction = correction_set[correction_name]
    
    #Given the pt and eta edges, loop over their midvalues.
    for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
        for pt_low, pt_high in zip(pt_edges[:-1], pt_edges[1:]):
            for rho_low, rho_high in zip(rho_edges[:-1], rho_edges[1:]):
            
                eta = (eta_low + eta_high) / 2
                pt  = (pt_low  + pt_high) / 2
                rho  = (rho_low  + rho_high) / 2
                values = [eta, pt, rho]
                sf = correction.evaluate(*values)
                
                scale_factors.append({
                    'campaign': campaign,
                    'eta_low' : eta_low,
                    'eta_high': eta_high, 
                    'pt_low'  : pt_low,
                    'pt_high' : pt_high,
                    'rho_low'  : rho_low,
                    'rho_high' : rho_high,
                    'sf'      : sf,
                })
    
    df = pd.DataFrame(scale_factors)
    return df

print('Function for jets loaded.')      

Function for jets loaded.


### b-tagging / mis-tagging efficiency

In [6]:
def parse_bjet_eff(filename, correction_name, campaign):

    scale_factors=[]
    with open(filename, "r") as f: json_data = json.load(f)

    pt_edges     = [20.0, 30.0, 50.0, 70.0, 100.0, 140.0, 200.0, 300.0, 600.0, 1000.0] #Taken from the input file
    eta_edges = [0, 2.5] #Taken from the input file
    flavors = [0, 5, 4]

    correction_set = correctionlib.CorrectionSet.from_file(filename)
    for flav in flavors:
        for eta_low, eta_high in zip(eta_edges[:-1], eta_edges[1:]):
            for pt_low, pt_high in zip(pt_edges[:-1], pt_edges[1:]):
                
                eta = (eta_low + eta_high) / 2
                pt  = (pt_low  + pt_high) / 2
                wp = 'M'
                if flav in [4, 5]: correction = correction_set[correction_name] #For b and c jets
                else: correction = correction_set['deepJet_incl'] #for light jets

                values      = ['central', 'M', flav, eta, pt]
                values_up   = ['up', 'M', flav, eta, pt]
                values_down = ['down', 'M', flav, eta, pt]
                #Options: central, up, up_correlated, up_uncorrelated, down, down_correlated, down_uncorrelated

                sf     = correction.evaluate(*values)
                sfup   = correction.evaluate(*values_up)
                sfdown = correction.evaluate(*values_down)
                
                scale_factors.append({
                    'campaign': campaign,
                    'eta_low' : eta_low,
                    'eta_high': eta_high, 
                    'pt_low'  : pt_low,
                    'pt_high' : pt_high,
                    'flav'    : flav,
                    'sfdown'  : sfdown,
                    'sf'      : sf,
                    'sfup'    : sfup
                })   
    
    df = pd.DataFrame(scale_factors)
    return df

print('Function for b-jets loaded.')

Function for b-jets loaded.


## Main: Iterating over the object dictionary to find scale-factors for each

In [7]:
%%time

for obj, val in objectdict.items():
    
    #if obj not in ['Electron']: continue ### For testing purposes
    #if obj not in ['Jet-JEC', 'Jet-JER', 'Jet-pT']: continue ### For testing purposes

    print(f'\n'+'-'*50+f'\n\033[032mProcessing corrections for: {obj}\033[0m\n'+'-'*50)
    basedir = val['basedir']
    outdir = val['outdir']
    os.makedirs(outdir, exist_ok=True)
    campaigns = os.listdir(basedir) #list only folders, not files
    files = []
    for camp in campaigns:
        if camp not in ['2018_UL', '2017_UL', '2016preVFP_UL', '2016postVFP_UL']: continue
        filename = os.path.join(basedir, camp, val['jsonfile'])
        if os.path.exists(filename) and filename.endswith('.json'):
            files.append((filename, camp))
    
    for correction in val['corrections']:
        print(f"\n\033[033mProcessing correction: {correction}\033[0m\n")
        output_filename = val['corrections'][correction]
        os.makedirs(val['outdir'], exist_ok=True)

        data = []
        for filename, campaign in files:
            print(f'Opening file: {filename} for correction: {correction}')
            if obj == 'Electron': extracted_data = parse_electron_sf(filename, correction, campaign)
            elif obj == 'Muon':   extracted_data = parse_muon_sf(filename, correction, campaign)
            elif obj == 'bJet':   extracted_data = parse_bjet_eff(filename, correction, campaign)
            elif obj =='Jet-JEC':
                if correction == 'Summer19UL18_V5_MC_Total_AK4PFchs':
                    if   '2016preVFP'  in campaign: correction = 'Summer19UL16APV_V7_MC_Total_AK4PFchs'
                    elif '2016postVFP' in campaign: correction = 'Summer19UL16_V7_MC_Total_AK4PFchs'
                    elif '2017'        in campaign: correction = 'Summer19UL17_V5_MC_Total_AK4PFchs'
                    extracted_data = parse_jet_jec_sf(filename, correction, campaign)
                correction = 'Summer19UL18_V5_MC_Total_AK4PFchs' ### return to previous so that this loop can happen again.
            elif obj == 'Jet-JER':
                if correction == 'Summer19UL18_JRV2_MC_ScaleFactor_AK4PFchs':
                    if   '2016preVFP'  in campaign: correction = 'Summer20UL16APV_JRV3_MC_ScaleFactor_AK4PFchs'
                    elif '2016postVFP' in campaign: correction = 'Summer20UL16_JRV3_MC_ScaleFactor_AK4PFchs'
                    elif '2017'        in campaign: correction = 'Summer19UL17_JRV2_MC_ScaleFactor_AK4PFchs'
                    extracted_data = parse_jet_jer_sf(filename, correction, campaign)
                correction = 'Summer19UL18_JRV2_MC_ScaleFactor_AK4PFchs'  ### return to previous so that this loop can happen again.
            elif obj == 'Jet-pT':
                if correction == 'Summer19UL18_JRV2_MC_PtResolution_AK4PFchs':
                    if   '2016preVFP'  in campaign: correction = 'Summer20UL16APV_JRV3_MC_PtResolution_AK4PFchs'
                    elif '2016postVFP' in campaign: correction = 'Summer20UL16_JRV3_MC_PtResolution_AK4PFchs'
                    elif '2017'        in campaign: correction = 'Summer19UL17_JRV2_MC_PtResolution_AK4PFchs'
                    extracted_data = parse_jet_ptres_sf(filename, correction, campaign)
                correction = 'Summer19UL18_JRV2_MC_PtResolution_AK4PFchs' ### return to previous so that this loop can happen again.                    
                
            data.append(extracted_data)
            
        data = pd.concat(data, ignore_index=True)
        columns_to_round = ['sfdown', 'sf', 'sfup']
        existing_columns = [col for col in columns_to_round if col in data.columns]  ### Filter existing columns
        if data.empty:
            warning(f'Dataframe empty. Skipping correction: {correction}')
            continue
        data[existing_columns] = data[existing_columns].round(6)
        display(pd.concat([data.head(2), data.tail(2)]))

        outfile = os.path.join(val['outdir'], output_filename)    
        with open(outfile, 'w') as f:
            for index, row in data.iterrows():
                ### Text formatting:
                formatted_row = ""
                for i, column in enumerate(data.columns):
                    if i == 0:                       formatted_row += f"{str(row[column]):<20}"
                    elif column in columns_to_round: formatted_row += f"{str(row[column]):<12}"
                    else:                            formatted_row += f"{str(row[column]):<8}"
                f.write(formatted_row.strip() + "\n")

        print(f"Data written to {outfile}")

print('\nDone!\n')


--------------------------------------------------
Processing corrections for: Electron
--------------------------------------------------

Processing correction: UL-Electron-ID-SF

Opening file: POG/EGM/2016postVFP_UL/electron.json for correction: UL-Electron-ID-SF
Extracting data for: 2016postVFP
Extracting data for:sf
Extracting data for Medium WP
Edges extracted!
Correctionlib evaluated and dataframe created.

Opening file: POG/EGM/2016preVFP_UL/electron.json for correction: UL-Electron-ID-SF
Extracting data for: 2016preVFP
Extracting data for:sf
Extracting data for Medium WP
Edges extracted!
Correctionlib evaluated and dataframe created.

Opening file: POG/EGM/2017_UL/electron.json for correction: UL-Electron-ID-SF
Extracting data for: 2017
Extracting data for:sf
Extracting data for Medium WP
Edges extracted!
Correctionlib evaluated and dataframe created.

Opening file: POG/EGM/2018_UL/electron.json for correction: UL-Electron-ID-SF
Extracting data for: 2018
Extracting data for:s

,campaign,eta_low,eta_high,pt_low,pt_high,sfdown,sf,sfup
0,2016postVFP_UL,-2.5,-2.0,10.0,20.0,1.024601,1.039711,1.054822
1,2016postVFP_UL,-2.5,-2.0,20.0,35.0,0.999609,1.015896,1.032183
218,2018_UL,2.0,2.5,100.0,200.0,0.972655,0.985459,0.998263
219,2018_UL,2.0,2.5,200.0,1500.0,0.881451,0.933747,0.986044


Data written to corrections/electronsf/electron_id_sf.txt

--------------------------------------------------
Processing corrections for: Muon
--------------------------------------------------

Processing correction: NUM_MediumID_DEN_genTracks

Opening file: POG/MUO/2016postVFP_UL/muon_Z_v2.json for correction: NUM_MediumID_DEN_genTracks
Extracting data for: 2016postVFP_UL
Edges extracted!
Opening file: POG/MUO/2016preVFP_UL/muon_Z_v2.json for correction: NUM_MediumID_DEN_genTracks
Extracting data for: 2016preVFP_UL
Edges extracted!
Opening file: POG/MUO/2017_UL/muon_Z_v2.json for correction: NUM_MediumID_DEN_genTracks
Extracting data for: 2017_UL
Edges extracted!
Opening file: POG/MUO/2018_UL/muon_Z_v2.json for correction: NUM_MediumID_DEN_genTracks
Extracting data for: 2018_UL
Edges extracted!


,campaign,eta_low,eta_high,pt_low,pt_high,sfdown,sf,sfup
0,2016postVFP_UL,0.0,0.9,15.0,20.0,0.769809,1.063897,1.357984
1,2016postVFP_UL,0.0,0.9,20.0,25.0,0.953175,0.991884,1.030593
126,2018_UL,2.1,2.4,60.0,120.0,0.920821,0.944989,0.969157
127,2018_UL,2.1,2.4,120.0,1500.0,0.920821,0.944989,0.969157


Data written to corrections/muonsf/muon_id_sf.txt

Processing correction: NUM_TightRelIso_DEN_MediumID

Opening file: POG/MUO/2016postVFP_UL/muon_Z_v2.json for correction: NUM_TightRelIso_DEN_MediumID
Extracting data for: 2016postVFP_UL
Edges extracted!
Opening file: POG/MUO/2016preVFP_UL/muon_Z_v2.json for correction: NUM_TightRelIso_DEN_MediumID
Extracting data for: 2016preVFP_UL
Edges extracted!
Opening file: POG/MUO/2017_UL/muon_Z_v2.json for correction: NUM_TightRelIso_DEN_MediumID
Extracting data for: 2017_UL
Edges extracted!
Opening file: POG/MUO/2018_UL/muon_Z_v2.json for correction: NUM_TightRelIso_DEN_MediumID
Extracting data for: 2018_UL
Edges extracted!


...

Data written to corrections/jetsf/jet_ptres_sf.txt

--------------------------------------------------
Processing corrections for: bJet
--------------------------------------------------

Processing correction: deepJet_mujets

Opening file: POG/BTV/2016postVFP_UL/btagging.json for correction: deepJet_mujets
Opening file: POG/BTV/2016preVFP_UL/btagging.json for correction: deepJet_mujets
Opening file: POG/BTV/2017_UL/btagging.json for correction: deepJet_mujets
Opening file: POG/BTV/2018_UL/btagging.json for correction: deepJet_mujets


,campaign,eta_low,eta_high,pt_low,pt_high,flav,sfdown,sf,sfup
0,2016postVFP_UL,0,2.5,20.0,30.0,0,0.726181,0.863539,1.000897
1,2016postVFP_UL,0,2.5,30.0,50.0,0,0.615162,0.732491,0.849819
106,2018_UL,0,2.5,300.0,600.0,4,0.735934,0.954257,1.172580
107,2018_UL,0,2.5,600.0,1000.0,4,0.749219,0.965463,1.181707


Data written to corrections/bjeteff/bjet_mujets_and_incl_eff.txt

Processing correction: deepJet_comb

Opening file: POG/BTV/2016postVFP_UL/btagging.json for correction: deepJet_comb
Opening file: POG/BTV/2016preVFP_UL/btagging.json for correction: deepJet_comb
Opening file: POG/BTV/2017_UL/btagging.json for correction: deepJet_comb
Opening file: POG/BTV/2018_UL/btagging.json for correction: deepJet_comb


,campaign,eta_low,eta_high,pt_low,pt_high,flav,sfdown,sf,sfup
0,2016postVFP_UL,0,2.5,20.0,30.0,0,0.726181,0.863539,1.000897
1,2016postVFP_UL,0,2.5,30.0,50.0,0,0.615162,0.732491,0.849819
106,2018_UL,0,2.5,300.0,600.0,4,0.828379,0.930683,1.032986
107,2018_UL,0,2.5,600.0,1000.0,4,0.684487,0.880405,1.076323


Data written to corrections/bjeteff/bjet_comb_and_incl_eff.txt

Done!

CPU times: user 19.5 s, sys: 336 ms, total: 19.8 s
Wall time: 22.5 s
